In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# File paths
rh_path = "data/DonneesRH.xlsx"
sport_path = "data/DonneesSportive.xlsx"

# Load RH data (you may need to specify the sheet name if there's more than one)
df_rh = pd.read_excel(rh_path)
df_sport = pd.read_excel(sport_path)

merged_df = pd.merge(df_rh, df_sport, on="ID salarié", how="inner")


In [ ]:
merged_df.head()

In [ ]:
print(len(df_sport))
print(len(df_rh))

sport_not_in_rh = set(df_sport["ID salarié"]) - set(df_rh["ID salarié"])
print("IDs in Sport but missing in RH:", sport_not_in_rh)

rh_not_in_sport = set(df_rh["ID salarié"]) - set(df_sport["ID salarié"])
print("IDs in RH but missing in Sport:", rh_not_in_sport)


In [ ]:
# Check for nulls
print(df_rh.isnull().sum())
print(df_sport.isnull().sum())

# Check unique values

print(df_sport["Pratique d'un sport"].unique())


In [ ]:
print(df_rh['Moyen de déplacement'].unique())

In [ ]:
import json

try:
    with open("secret.json") as f:
        secrets = json.load(f)
    API_KEY = secrets.get("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("API key not found in secret.json")
except FileNotFoundError:
    raise FileNotFoundError("secret.json not found. Please add your API key.")


In [ ]:
company_Adress = "1362 Av. des Platanes, 34970 Lattes"
employee_Adress = "128 Rue du Port, 34000 Frontignan"
employee_Adress2 = "74 Rue des Fleurs, 34970 Lattes"

In [ ]:
import googlemaps
from datetime import datetime
from typing import Optional, Dict, List, Tuple
import functools

# Cache decorator for API calls
def cache_results(func):
    """Simple memoization decorator to cache API responses"""
    cache = {}
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        cache_key = str(args) + str(kwargs)
        if cache_key not in cache:
            cache[cache_key] = func(*args, **kwargs)
        return cache[cache_key]
    return wrapper

@cache_results
def geocode_address(client: googlemaps.Client, address: str) -> Optional[Tuple[float, float]]:
    """Convert address to coordinates using Geocoding API"""
    try:
        result = client.geocode(address)
        if result:
            location = result[0]['geometry']['location']
            return (location['lat'], location['lng'])
    except Exception as e:
        print(f"Geocoding failed for '{address}': {str(e)}")
    return None

def calculate_route(
    client: googlemaps.Client,
    origin: str,
    destination: str,
    mode: str = "driving",
    alternatives: bool = False,
    units: str = "metric"
) -> Optional[Dict]:
    """Calculate route between two locations using Routes API"""
    try:
        directions = client.directions(
            origin,
            destination,
            mode=mode,
            departure_time=datetime.now(),
            alternatives=alternatives,
            units=units
        )
        
        if not directions:
            return None
            
        primary_route = directions[0]
        leg = primary_route['legs'][0]
        
        result = {
            "origin": leg['start_address'],
            "destination": leg['end_address'],
            "distance": leg['distance']['text'],
            "duration": leg['duration']['text'],
            "travel_mode": mode.upper(),
            "steps": [{
                "instruction": step['html_instructions'],
                "distance": step['distance']['text'],
                "duration": step['duration']['text']
            } for step in leg['steps']]
        }
        
        if alternatives and len(directions) > 1:
            result["alternatives"] = [{
                "summary": alt['summary'],
                "distance": alt['legs'][0]['distance']['text'],
                "duration": alt['legs'][0]['duration']['text']
            } for alt in directions[1:]]
        
        return result
        
    except Exception as e:
        print(f"Route calculation failed: {str(e)}")
        return None

def get_route_between_addresses(
    client: googlemaps.Client,
    origin_address: str,
    destination_address: str,
    **kwargs
) -> Optional[Dict]:
    """Higher-level function that handles geocoding and routing"""
    origin_coords = geocode_address(client, origin_address)
    destination_coords = geocode_address(client, destination_address)
    
    if not origin_coords or not destination_coords:
        return None
        
    return calculate_route(
        client,
        origin=origin_coords,
        destination=destination_coords,
        **kwargs
    )

# Example usage
if __name__ == "__main__":
    # Initialize client once
    client = googlemaps.Client(key=API_KEY)
    
    # Example 1: Direct route calculation
    route = calculate_route(
        client,
        origin=employee_Adress,
        destination=company_Adress,
        mode="walking"
    )
    
    if route:
        print(f"Route from {route['origin']} to {route['destination']}")
        print(f"Distance: {route['distance']}")
        print(f"Duration: {route['duration']}")
        print(f"First step: {route['steps'][0]['instruction']}")
    
    company_Adress = "1362 Av. des Platanes, 34970 Lattes"
    employee_Adress = "128 Rue du Port, 34000 Frontignan"
    employee_Adress2 = "74 Rue des Fleurs, 34970 Lattes"

    # Example 2: With automatic geocoding
    route = get_route_between_addresses(
        client,
        origin_address=employee_Adress,
        destination_address=company_Adress,
        mode="driving",
        alternatives=True
    )
    
    if route:
        print(f"\nDriving route from {route['origin']} to {route['destination']}")
        print(f"Distance: {route['distance']}")
        print(f"Duration: {route['duration']}")
        if "alternatives" in route:
            print(f"\nFound {len(route['alternatives'])} alternative routes:")
            for i, alt in enumerate(route['alternatives'], 1):
                print(f"{i}. {alt['summary']} ({alt['distance']}, {alt['duration']})")

In [ ]:
import pandas as pd
from tqdm import tqdm
import time
import random
from typing import Optional, Dict

# Initialize the client once
client = googlemaps.Client(key=API_KEY)

# Your company address (set this variable)
company_address = "Your Company Full Address, City, Postal Code, Country"

def get_employee_commute(row: pd.Series, company_addr: str) -> Optional[Dict]:
    """Calculate commute for a single employee"""
    # Map transportation modes to Google Maps modes
    transport_mapping = {
        'Transports en commun': 'transit',
        'véhicule thermique/électrique': 'driving',
        'Marche/running': 'walking',
        'Vélo/Trottinette/Autres': 'bicycling'
    }
    
    try:
        mode = transport_mapping.get(row['Moyen de déplacement'], 'driving')
        
        # Get route using your existing function
        result = get_route_between_addresses(
            client=client,
            origin_address=row['Adresse du domicile'],
            destination_address=company_addr,
            mode=mode
        )
        
        if result:
            return {
                'Distance_km': float(result['distance'].split()[0].replace(',', '.')),  # Extract numeric value
                'Duree_min': int(result['duration'].split()[0])  # Extract minutes
            }
            
    except Exception as e:
        print(f"Error processing {row['Nom']} {row['Prénom']}: {str(e)}")
    
    return None

# Add progress bar
tqdm.pandas(desc="Calcul des trajets domicile-travail")

# Apply to each row with error handling and rate limiting
commute_data = []
for _, row in tqdm(df_rh.iterrows(), total=len(df_rh)):
    commute = get_employee_commute(row, company_address)
    commute_data.append(commute)
    time.sleep(random.uniform(0.7, 1.3))  # Gentle rate limiting

# Add results to DataFrame
df_rh['Distance_km'] = [x['Distance_km'] if x else None for x in commute_data]
df_rh['Duree_min'] = [x['Duree_min'] if x else None for x in commute_data]

# Show results
print(df_rh[['Nom', 'Prénom', 'Adresse du domicile', 'Moyen de déplacement', 'Distance_km', 'Duree_min']].head())

In [ ]:
a = "128 Rue du Port, 34000 Frontignan"

route = get_route_between_addresses(
    client,
    origin_address=a,
    destination_address=company_Adress,
    mode="transit",
    alternatives=False
    )

In [ ]:
route

In [ ]:
OUTPUT_CSV = "employee_commutes.csv"

df_rh.to_csv(OUTPUT_CSV, index=False)

print("\nProcessing complete! Results saved to", OUTPUT_CSV)
print(df_rh[['Nom', 'Prénom', 'Distance_km', 'Duree_min']].head())